In [1]:
import pandas as pd

In [2]:
#import schools csv
school_file="Resources/schools_complete.csv"
school_df=pd.read_csv(school_file)
#import students csv
student_csv="Resources/students_complete.csv"
student_df=pd.read_csv(student_csv)

In [3]:
#District Summary
summary_df= pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
#summary_df.head()

In [4]:
total_schools=school_df["School ID"].count()
total_students=student_df["Student ID"].count()
total_budget = school_df["budget"].sum()
math_average = student_df["math_score"].mean()
reading_average = student_df["reading_score"].mean()


In [5]:
#make pass and fail df
pass_fail_df = student_df[["student_name","reading_score","math_score"]]
reading_df= pass_fail_df.loc[pass_fail_df["reading_score"]>=70,["student_name","reading_score"]]
math_df = pass_fail_df.loc[pass_fail_df["math_score"]>=70,["student_name","math_score","reading_score"]]
both_df = math_df.loc[math_df["reading_score"]>=70,["student_name","math_score","reading_score"]]

#count pass
math_pass = math_df["student_name"].count()
reading_pass = reading_df["student_name"].count()
both_pass = both_df["student_name"].count()

#get percent
math_pass_percent = round((math_pass/total_students)*100,2)
reading_pass_percent = round((reading_pass/total_students)*100,2)
both_pass_percent = round((both_pass/total_students)*100,2)

In [6]:
district_summary_df=pd.DataFrame({"Total Schools":total_schools,
                        "Total Students":total_students,
                         "Total Budget":total_budget,
                         "Average Math Score":f"% {round(math_average,2)}",
                         "Average Reading Score":f"% {round(reading_average,2)}",
                         "Percent passing math":math_pass_percent,
                         "Percent passing reading":reading_pass_percent,
                         "Percent passing overall":both_pass_percent
                        }, index=["Table"])
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent passing math,Percent passing reading,Percent passing overall
Table,15,"39,170","$24,649,428",% 78.99,% 81.88,74.98,85.81,65.17


In [7]:
school_summary_df= school_df[["school_name","type","size","budget"]]

school_summary_df["Per Student Budget"] = school_summary_df["budget"]/school_summary_df["size"]

In [8]:
school_groups=student_df.groupby(student_df["school_name"])
school_math_average=round(school_groups["math_score"].mean(),2)
school_reading_average=round(school_groups["reading_score"].mean(),2)
averages_df=pd.merge(school_math_average,school_reading_average, on = "school_name")
school_summary_df=pd.merge(school_summary_df,averages_df, on = "school_name")

In [9]:
#Make pass fail bins for each student

bin_data_df= student_df

bins = [0,69.99,101]

group_names = ["Fail","Pass"]

bin_data_df["Math"] = pd.cut(bin_data_df["math_score"], bins, labels=group_names, include_lowest=True)
bin_data_df["Reading"] = pd.cut(bin_data_df["reading_score"], bins, labels=group_names, include_lowest=True)

In [10]:
school_pass_fail_math= bin_data_df.groupby([bin_data_df["school_name"],bin_data_df["Math"]]).size()
school_pass_fail_math
school_math_df = school_pass_fail_math.to_frame(name = '% Passing Math').reset_index()
school_totals_df = school_df[["school_name","size"]]
school_math_df = pd.merge(school_math_df, school_totals_df, on="school_name")
school_math_df["% Passing Math"]=(school_math_df["% Passing Math"]/school_math_df["size"])*100
passingm_df=school_math_df.loc[school_math_df["Math"]=="Pass",["school_name","% Passing Math"]]
school_summary_df=pd.merge(school_summary_df,passingm_df, on = "school_name")

school_pass_fail_reading= bin_data_df.groupby([bin_data_df["school_name"],bin_data_df["Reading"]]).size()
school_pass_fail_reading
school_reading_df = school_pass_fail_reading.to_frame(name = "% Passing Reading").reset_index()
school_totals_df = school_df[["school_name","size"]]
school_reading_df = pd.merge(school_reading_df, school_totals_df, on="school_name")
school_reading_df["% Passing Reading"]=(school_reading_df["% Passing Reading"]/school_reading_df["size"])*100
passingr_df=school_reading_df.loc[school_reading_df["Reading"]=="Pass",["school_name","% Passing Reading"]]
school_summary_df=pd.merge(school_summary_df,passingr_df, on = "school_name")

school_pass_fail_both= bin_data_df.groupby([bin_data_df["school_name"],bin_data_df["Math"],bin_data_df["Reading"]]).size()
school_pass_fail_both
school_both_df = school_pass_fail_both.to_frame(name = '% Passing Both').reset_index()
school_totals_df = school_df[["school_name","size"]]
school_both_df = pd.merge(school_both_df, school_totals_df, on="school_name")
school_both_df["% Passing Both"]=(school_both_df["% Passing Both"]/school_both_df["size"])*100
passingb1_df=school_both_df.loc[school_both_df["Math"]=="Pass",:]
passingb2_df=passingb1_df.loc[passingb1_df["Reading"]=="Pass",["school_name","% Passing Both"]]
school_summary_df=pd.merge(school_summary_df,passingb2_df, on = "school_name")

spending_df=school_summary_df
size_df=school_summary_df
type_df=school_summary_df

In [11]:
school_summary_df = school_summary_df.rename(columns={"school_name":"School Name",
                                                      "type":"School Type",
                                                      "size":"Total Students",
                                                      "budget":"Total School Budget",
                                                      "math_score":"Average Math Score",
                                                      "reading_score":"Average Reading Score"})


final_school_summary_df=school_summary_df

final_school_summary_df["Total School Budget"] = final_school_summary_df["Total School Budget"].map("${:,}".format)
final_school_summary_df["Total Students"] = final_school_summary_df["Total Students"].map("{:,}".format)
final_school_summary_df["Per Student Budget"] = final_school_summary_df["Per Student Budget"].map("${:.2f}".format)

final_school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
0,Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.683922,81.316421,53.513884
1,Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.988471,80.739234,53.204476
2,Shelton High School,Charter,"1,761","$1,056,600",$600.00,83.36,83.73,93.867121,95.854628,89.892107
3,Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.752967,80.862999,53.527508
4,Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.392371,97.138965,90.599455
5,Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.27,83.99,93.867718,96.539641,90.582567
6,Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.133477,97.039828,91.334769
7,Bailey High School,District,"4,976","$3,124,928",$628.00,77.05,81.03,66.680064,81.933280,54.642283
8,Holden High School,Charter,427,"$248,087",$581.00,83.80,83.81,92.505855,96.252927,89.227166
9,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.594595,95.945946,90.540541


In [12]:
Decending_Schools_df=school_summary_df.sort_values(by='% Passing Both', ascending=False)
Decending_Schools_df=Decending_Schools_df.set_index("School Name")
Top_Schools_df=Decending_Schools_df.head(5)


In [13]:
Bottom_Schools_df=Decending_Schools_df.tail(5)
Bottom_Schools_df=Bottom_Schools_df.sort_values(by='% Passing Both', ascending=True)
Bottom_Schools_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637.00,76.84,80.74,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.057551,81.222432,53.539172


In [14]:
grade_groups_df=summary_df.groupby([summary_df["school_name"],summary_df["grade"]])
grade_math_df=grade_groups_df[["school_name","grade","math_score"]]
grade_math_df=grade_math_df.mean()
grade_math_df=grade_math_df.unstack(level=-1)
grade_math_df

math_score                                 
grade                       10th       11th       12th        9th
school_name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

In [15]:
grade_reading_df=grade_groups_df[["school_name","grade","reading_score"]]
grade_reading_df=grade_reading_df.mean()
grade_reading_df=grade_reading_df.unstack(level=-1)
grade_reading_df

reading_score                                 
grade                          10th       11th       12th        9th
school_name                                                         
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333

In [16]:
bins = [0,583.9,629.9,644.9,676]

bin_names = ["<$584","$585-629","$630-644","$645-675"]

spending_df["Spending Ranges (Per Student)"] = pd.cut(spending_df["Per Student Budget"], bins, labels=bin_names, include_lowest=True)
adjusted_spending_df=spending_df.drop(["school_name","type","size","budget"],axis=1)
final_spending_df=adjusted_spending_df.groupby(adjusted_spending_df["Spending Ranges (Per Student)"])
final_spending_df=final_spending_df.mean()
final_spending_df=final_spending_df.drop("Per Student Budget",axis=1)
final_spending_df

,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Both
Spending Ranges (Per Student),,,,,
<$584,83.452500,83.935000,93.460096,96.610877,90.369459
$585-629,81.900000,83.155000,87.133538,92.718205,81.418596
$630-644,78.517500,81.625000,73.484209,84.391793,62.857656
$645-675,76.996667,81.026667,66.164813,81.133951,53.526855


In [17]:
bins = [0,999,1999,5001]

bin_names = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]


size_df["School Size"] = pd.cut(spending_df["size"], bins, labels=bin_names, include_lowest=True)
adjusted_size_df=size_df.drop(["school_name","type","Per Student Budget","budget"],axis=1)
final_size_df=adjusted_size_df.groupby(adjusted_size_df["School Size"])
final_size_df=final_size_df.mean()
final_size_df=final_size_df.drop("size",axis=1)
final_size_df

,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Both
School Size,,,,,
Small(<1000),83.820,83.92500,93.550225,96.099437,89.883853
Medium(1000-2000),83.374,83.86800,93.599695,96.790680,90.621535
Large(2000-5000),77.745,81.34375,69.963361,82.766634,58.286003


In [18]:
final_type_df=type_df.groupby(type_df["type"])
final_type_df.mean()

,size,budget,Per Student Budget,math_score,reading_score,% Passing Math,% Passing Reading,% Passing Both
type,,,,,,,,
Charter,1524.250000,9.126881e+05,599.500000,83.472500,83.897500,93.620830,96.586489,90.432244
District,3853.714286,2.478275e+06,643.571429,76.955714,80.965714,66.548453,80.799062,53.672208
